## Feature Generation & Model
Lpay의 다음 결제까지 걸릴 시간을 예측한다.

### Import module

In [ ]:
import os
import numpy as np
import pandas as pd
import warnings ; warnings.filterwarnings(action='ignore')

# Feature Transformation
from sklearn.preprocessing import StandardScaler

# Model
from catboost import CatBoostRegressor

# Evaluate
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

# Save Model
import joblib

### Read Data

In [ ]:
data_path = "../3. 경서경욱_데이터 및 모델 세이브 파일/dataset/"
model_path = "../3. 경서경욱_데이터 및 모델 세이브 파일/model/"

lpay = pd.read_csv(data_path+'/LPOINT_BIG_COMP_06_LPAY.csv', parse_dates=['de_dt'])
lpay['date'] = pd.to_datetime(lpay['de_dt'].astype(str)+' '+lpay['de_hr'].astype(str).str.zfill(2))
lpay

### Data Cleansing

In [ ]:
# 월, 요일(문자), 주말
lpay['DE_M'] = lpay['de_dt'].dt.month
lpay["consum_day"] = lpay["de_dt"].dt.dayofweek.map({i:j for i, j in enumerate(["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"])})
lpay["weekend_whether"] = lpay["consum_day"].apply(lambda x : 1 if (x == "Sat") or (x == "Sun") else 0)

# 공휴일
holiday_list = ['2021-01-01', '2021-02-11', '2021-02-12', '2021-02-13', '2021-03-01',
                '2021-05-05', '2021-05-19', '2021-06-06', '2021-08-15', '2021-09-20',
                '2021-09-21', '2021-09-22', '2021-10-03', '2021-10-09', '2021-12-25']
lpay["holiday"] = lpay["de_dt"].apply(lambda x : 1 if x in holiday_list else 0)
lpay

### Feature Generation & Save Data

In [ ]:
trade_list = lpay.drop_duplicates(['cust','cop_c','date'])[['cust', 'date']]\
             .groupby('date')['cust'].unique().reset_index()

feature = pd.DataFrame()
for _, DATE, CUSTS in trade_list.itertuples():
    ease = lpay.query('cust in @CUSTS & date <= @DATE')
    # 실제 구매액, 구매건수, 평균구매액, 최대구매액
    amount = ease.groupby('cust')['buy_am'].agg([('real_pay', np.sum),('consum_count', np.size),
                                                 ('mean_pay', lambda x : np.round(np.mean(x))),('max_pay', np.max)])
    
    # 내점일수, 최근구매일, 구매주기
    day = ease.groupby('cust')['de_dt'].agg([('visit_day_count',lambda x: x.nunique()), 
                                             ('recent_consum_day', lambda x: (ease.de_dt.max() - x.max()).days), 
                                             ('consum_cycle', lambda x: int((x.max() - x.min()).days / x.nunique()))])
    
    # 1일 평균구매액, 1일 평균구매건수, 1일 1회 구매
    daily = ease.groupby(['cust', 'de_dt'])['buy_am'].agg([('day1_pay', np.sum),('day1_pay_count', np.size)]).reset_index()\
            .groupby('cust').agg({'day1_pay':[('day1_mean_pay', np.mean)],
                                  'day1_pay_count':[('day1_mean_pay_count', np.mean),
                                                    ('day1_1consum', lambda x: x.tolist().count(1))]}) 
    daily.columns = daily.columns.get_level_values(1)
    
    # cop 별 구매비율, 총금액, 평균금액, 구매주기, cop 종류수, 표준편차, 
    cop = pd.concat([pd.pivot_table(ease, index='cust', columns='cop_c', values='buy_am', aggfunc=np.size, fill_value=0)\
                     .divide(ease.groupby('cust')['cop_c'].size(), axis=0),
                     pd.pivot_table(ease, index='cust', columns='cop_c', values='buy_am', aggfunc=np.sum, fill_value=0),
                     pd.pivot_table(ease, index='cust', columns='cop_c', values='buy_am', aggfunc=np.mean, fill_value=0),
                     pd.pivot_table(ease, index='cust', columns='cop_c', values='de_dt', 
                                    aggfunc=lambda x: (x.max()-x.min()).days//x.nunique(), fill_value=0),
                     ease.groupby("cust")["cop_c"].nunique(),
                     pd.pivot_table(ease, index='cust', columns='cop_c', values='buy_am', aggfunc=np.std, fill_value=0)], axis=1)
    cop.columns = [f'{j}_{i}' for i in ['ratio','total_amount','mean_amount','consum_cycle'] for j in sorted(ease.cop_c.unique())]\
                  +["cop_c_unique"]\
                  +[f'{i}_sd' for i in cop.columns[ease.cop_c.nunique()*4+1:]]

    # 오프라인/온라인 건수, 온라인 비율
    onoff = pd.concat([pd.pivot_table(ease, index='cust', columns='chnl_dv', values='buy_am', aggfunc=np.size, fill_value=0)\
                       .rename(columns={1:'off_count', 2:'on_count'}),
                       ease.groupby("cust")["chnl_dv"].agg([("online_ratio", np.mean)])], axis=1)

    # 월평균구매액,월최대구매액,월최소구매액, 평균방문월, 평균구매월
    high = ease.groupby('cust')['buy_am'].agg([('high_rank_pay', lambda x: x.sort_values()[-5:].index.tolist())]) #상위구매액
    high = [j for i in high.high_rank_pay for j in i]
    month = pd.concat([pd.pivot_table(ease ,index='cust', columns='DE_M', values='buy_am',
                                      aggfunc = [np.mean, max, min], fill_value=0),
                       ease.groupby('cust')['DE_M'].mean(),
                       ease.loc[high].groupby('cust')['DE_M'].mean()], axis=1)
    month.columns = [f'{str(j)}_{i}' for i in ['month_mean_pay', 'month_max_pay', 'month_min_pay'] for j in sorted(ease['DE_M'].unique())]\
                    +['mean_visit_month', 'mean_consum_month']

    # 요일 종류수, 공휴일 비율, 주말 비율
    day_info = pd.concat([ease.groupby("cust")["consum_day"].agg([("consum_day_unique", pd.Series.nunique)]),
                          ease.groupby("cust")["holiday"].agg([("holiday_ratio", np.mean)]),
                          ease.groupby("cust")["weekend_whether"].agg([("weekend_whether_ratio", np.mean)])], axis=1)
    
    make_feature = pd.concat([pd.DataFrame({'date':[DATE]*len(CUSTS)}, index=CUSTS),
                              amount, day, daily, cop, onoff, month, day_info],axis=1).reset_index()
    feature = pd.concat([feature, make_feature])
    
feature = feature.rename(columns={'index':'cust'}).sort_values(['cust','date']).reset_index(drop=True)
feature

In [ ]:
feature.to_csv(data_path+'/feature.csv', index=False)

### Generate Target & Save Data

In [ ]:
# target을 만들고자 하며 마지막 구매의 다음 구매까지 걸리는 시간을 구할 수 없어 임의의 값을 삽입해 처리해 구한다.
# 분석 기간 이외의 2040-08-01의 값으로 처리한다.
target = pd.concat([feature[['cust','date']],
                    pd.DataFrame({'cust':feature.cust.unique(), 'date':['2040-08-01 00:00:00']*feature.cust.nunique()})])
target['date'] = pd.to_datetime(target['date'])
target = target.sort_values(['cust','date']).reset_index(drop=True)

In [ ]:
target['target'] = target['date'].diff().dt.total_seconds()/60/60

# 첫 거래일은 이전 거래부터 현 거래까지의 값을 알 수 없음으로 삭제한다.
target = target.iloc[1:].drop(target.query('target < 0').index)

# 1년 기간 안의 시간간격 최댓값 24*364보다 크면 임의의 값을 넣은 날의 target값이다.
# 즉 고객의 마지막 구매일로 마지막 구매일의 target은 예측하고자 하는 다음 구매시간으로 test data로 구분한다.
feature = pd.concat([feature, target.reset_index(drop=True)['target']], axis=1)
train = feature.query('target <= 24*364')
test = feature.query('target > 24*364') ; del test['target']

In [ ]:
train.to_csv(data_path+'/feature_train.csv', index=False)
test.to_csv(data_path+'/feature_test.csv', index=False)

### Feature Transformation

In [ ]:
# Drop unuseful columns
del train['cust'], train['date']
del test['cust'], test['date']

# Define data
y_train = train['target'] ; del train['target']
X_train, X_test = train, test

In [ ]:
# Inputation
X_train.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)

In [ ]:
# Scaling
scaler = StandardScaler()
X_train[X_train.columns] =scaler.fit_transform(X_train) 
X_test[X_test.columns] = scaler.transform(X_test) 

### Modeling &  Save Model

In [ ]:
kf = KFold(n_splits = 10, shuffle = True, random_state = 1004)

In [ ]:
model = CatBoostRegressor(iterations=1000, learning_rate = 0.03, bootstrap_type ='Bayesian', 
                          devices='0:1', task_type="GPU", random_state=1004)

In [ ]:
rmse_list =[]
for tr_idx, val_idx in kf.split(np.array(X_train)):
    tr_x, tr_y = X_train.iloc[tr_idx], y_train.iloc[tr_idx]
    val_x, val_y = X_train.iloc[val_idx], y_train.iloc[val_idx]
    
    model.fit(tr_x, tr_y)
    pred = model.predict(val_x)
    rmse_list.append(mean_squared_error(val_y, pred)**0.5)
    
print(f'{model.__class__.__name__}의 10fold 평균 RMSE는 {np.mean(rmse_list)}이다.')

In [ ]:
# 일 단위로 변환 
print(np.mean(rmse_list)/24)

In [ ]:
joblib.dump(model, model_path + 'catboost_Lpay.pkl')